In [1]:
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.model_selection import cross_validate,KFold, RandomizedSearchCV
from sklearn.model_selection import train_test_split
import seaborn as sns
from math import pi
import matplotlib.pyplot as plt
import random
%matplotlib inline
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate,KFold, RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from xgboost import XGBClassifier

train_labels = pd.read_csv('Archivos de datos/train_labels.csv', encoding='latin-1')
train_values = pd.read_csv('Archivos de datos/train_values.csv', encoding='latin-1')

join = train_values.merge(train_labels)
join = join.astype({'land_surface_condition':'category', 'foundation_type':'category',\
                                    'roof_type':'category', 'ground_floor_type':'category',\
                                    'other_floor_type':'category', 'position':'category',\
                                    'plan_configuration':'category', 'legal_ownership_status':'category'
                                    })
## Experimentacion

join.columns
join['damage_grade'].value_counts()
X=pd.get_dummies(join.loc[:,:'has_secondary_use_other'])
y=join['damage_grade'].astype(int)

## Creamos set de entrenamiento y test

from sklearn.model_selection import train_test_split

In [4]:
join.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
0,802906,6,487,12198,2,30,6,5,t,r,...,0,0,0,0,0,0,0,0,0,3
1,28830,8,900,2812,2,10,8,7,o,r,...,0,0,0,0,0,0,0,0,0,2
2,94947,21,363,8973,2,10,5,5,t,r,...,0,0,0,0,0,0,0,0,0,3
3,590882,22,418,10694,2,10,6,5,t,r,...,0,0,0,0,0,0,0,0,0,2
4,201944,11,131,1488,3,30,8,9,t,r,...,0,0,0,0,0,0,0,0,0,3


In [5]:
join.columns

Index(['building_id', 'geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id',
       'count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage',
       'land_surface_condition', 'foundation_type', 'roof_type',
       'ground_floor_type', 'other_floor_type', 'position',
       'plan_configuration', 'has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'legal_ownership_status', 'count_families', 'has_secondary_use',
       'has_secondary_use_agriculture', 'has_secondary_use_hotel',
       'has_secondary_use_rental', 'has_secondary_use_institution',
       'has_secondary_use_school', 'has_secondary_use_i

In [2]:
#Seteo los rangos // Falta cambiar por Grid
n_estimators=[500, 600, 700]
learning_rate=[0.1,0.05,0.01]
max_depth=[5, 10, 15, 20]
gamma=[0, 0.5, 1, 1.5, 2, 5]
reg_lambda=[0, 1, 5, 10]
subsample=[0.2, 0.4, 0.6, 0.8, 1]
colsample_bytree=[0, 0.3, 0.5, 0.7, 1]
min_child_weight= [1, 5, 10]

param={'n_estimators':n_estimators,'max_depth':max_depth,'learning_rate':learning_rate,'gamma':gamma, 'reg_lambda':reg_lambda, 'subsample':subsample, 'colsample_bytree':colsample_bytree, 'min_child_weight':min_child_weight}

In [3]:
%%time

clf=XGBClassifier()
kf=KFold(n_splits=3,shuffle=True)
rs=RandomizedSearchCV(clf,param_distributions=param,cv=kf,scoring='f1_micro')
rs.fit(X,y)

C:\Users\ezequ\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[00:31:46] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\ezequ\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[00:35:03] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\ezequ\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

C:\Users\ezequ\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[02:42:45] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\ezequ\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[02:43:50] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Wall time: 2h 16min 26s


RandomizedSearchCV(cv=KFold(n_splits=3, random_state=None, shuffle=True),
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=...
                                           scale_pos_weight=None,
                                           subsample=None, tree_method=None,
                                           validate_parameters=None,
                                           verbosity=No

In [4]:
rs.best_params_

{'subsample': 0.6,
 'reg_lambda': 5,
 'n_estimators': 500,
 'min_child_weight': 1,
 'max_depth': 15,
 'learning_rate': 0.1,
 'gamma': 0.5,
 'colsample_bytree': 0.3}

In [ ]:
%%time

clf=XGBClassifier(n_estimators= 500, max_depth= 15,learning_rate= 0.1, colsample_bytree= 0.3, gamma= 0.5, min_child_weight= 1, reg_lambda= 5, subsample= 0.6)
clf.fit(X,y)

[12:23:11] INFO: ..\src\tree\updater_prune.cc:101: tree pruning end, 1274 extra nodes, 1236 pruned nodes, max_depth=15
[12:23:11] INFO: ..\src\tree\updater_prune.cc:101: tree pruning end, 1182 extra nodes, 1236 pruned nodes, max_depth=15
[12:23:11] INFO: ..\src\tree\updater_prune.cc:101: tree pruning end, 1256 extra nodes, 2132 pruned nodes, max_depth=15
[12:23:11] INFO: ..\src\tree\updater_prune.cc:101: tree pruning end, 268 extra nodes, 634 pruned nodes, max_depth=15
[12:23:11] INFO: ..\src\tree\updater_prune.cc:101: tree pruning end, 484 extra nodes, 1104 pruned nodes, max_depth=15
[12:23:11] INFO: ..\src\tree\updater_prune.cc:101: tree pruning end, 1742 extra nodes, 1698 pruned nodes, max_depth=15
[12:23:11] INFO: ..\src\tree\updater_prune.cc:101: tree pruning end, 608 extra nodes, 1178 pruned nodes, max_depth=15
[12:23:12] INFO: ..\src\tree\updater_prune.cc:101: tree pruning end, 4094 extra nodes, 2572 pruned nodes, max_depth=15
[12:23:12] INFO: ..\src\tree\updater_prune.cc:101: t

Exception ignored on calling ctypes callback function: <function _log_callback at 0x000002177EE31550>
Traceback (most recent call last):
  File "C:\Users\ezequ\anaconda3\lib\site-packages\xgboost\core.py", line 119, in _log_callback
    def _log_callback(msg):
KeyboardInterrupt: 


[12:23:14] INFO: ..\src\tree\updater_prune.cc:101: tree pruning end, 1110 extra nodes, 912 pruned nodes, max_depth=15
[12:23:14] INFO: ..\src\tree\updater_prune.cc:101: tree pruning end, 176 extra nodes, 530 pruned nodes, max_depth=15
[12:23:14] INFO: ..\src\tree\updater_prune.cc:101: tree pruning end, 344 extra nodes, 778 pruned nodes, max_depth=15
[12:23:15] INFO: ..\src\tree\updater_prune.cc:101: tree pruning end, 212 extra nodes, 610 pruned nodes, max_depth=15
[12:23:15] INFO: ..\src\tree\updater_prune.cc:101: tree pruning end, 512 extra nodes, 494 pruned nodes, max_depth=15
[12:23:15] INFO: ..\src\tree\updater_prune.cc:101: tree pruning end, 1160 extra nodes, 1196 pruned nodes, max_depth=15
[12:23:15] INFO: ..\src\tree\updater_prune.cc:101: tree pruning end, 3166 extra nodes, 2868 pruned nodes, max_depth=15
[12:23:15] INFO: ..\src\tree\updater_prune.cc:101: tree pruning end, 398 extra nodes, 956 pruned nodes, max_depth=15
[12:23:15] INFO: ..\src\tree\updater_prune.cc:101: tree pru

In [3]:
X_test=pd.read_csv('Archivos de datos/test_values.csv')
X_test.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,300051,17,596,11307,3,20,7,6,t,r,...,0,0,0,0,0,0,0,0,0,0
1,99355,6,141,11987,2,25,13,5,t,r,...,1,0,0,0,0,0,0,0,0,0
2,890251,22,19,10044,2,5,4,5,t,r,...,0,0,0,0,0,0,0,0,0,0
3,745817,26,39,633,1,0,19,3,t,r,...,0,0,1,0,0,0,0,0,0,0
4,421793,17,289,7970,3,15,8,7,t,r,...,0,0,0,0,0,0,0,0,0,0


In [4]:
prediction=clf.predict((pd.get_dummies(X_test)))
result=pd.DataFrame(prediction)

In [5]:
result['building_id']=X_test['building_id']
result.rename(columns={0:'damage_grade'},inplace=True)
result=result[['building_id','damage_grade']]
result.head()

,building_id,damage_grade
0,300051,3
1,99355,2
2,890251,2
3,745817,1
4,421793,3


In [6]:
result.to_csv('Archivos de datos/resultado.csv',index=False)